<a href="https://colab.research.google.com/github/mandie021/big-data-challenge/blob/main/level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>